$\Large\color{blue}{\text{PyGMTSAR 2017 Iran–Iraq Earthquake Co-Seismic Interferogram}}$

$\Large\color{blue}{\text{Sentinel-1 Scenes Downloading from Alaska Satellite Facility (ASF)}}$

$\large\color{blue}{\text{Compare the Results to GMTSAR and GAMMA Software}}$

### See and more PyGMTSAR notebooks on GitHub: [PyGMTSAR](https://github.com/mobigroup/gmtsar)

It works on

* MacOS Monterey (Apple Silicon) and BigSur (Intel) (Python 3.9) - please pre-install system dependencies (maybe using HomeBrew),

* Google Cloud VM and Notebooks on Debian 10 (Python 3.7), use the [Google Cloud Debian 10 init script](https://github.com/mobigroup/gmtsar/blob/master/gmtsar/sh/GMTSAR.install.debian10.sh) and [Google Cloud Debian 10 VM creation script](https://github.com/mobigroup/gmtsar/blob/master/gmtsar/sh/GMTSAR.gcloud_create_debian10.sh)

* $\color{red}{\text{Google Colab (Python 3.7) - you will be asked to re-run the notebook once due to "system crash" by desing}}$
$\color{red}{\text{Note: to open all notebook cells select menu "View" -> "Expand Sections"}}$

### PyGMTSAR is my free-time Open Source project.

That's a bit curious how the project was started a year ago. I develop geophysical inversion methods and processing software for many years using my fundamental physics and mathematics background. Satellite interferometry is the key point to validate my inversion models and I found the same problem as you too that the existing interferometry packages usage is a pain. There is no interactive processing with multiprocessing and even a progressbar and ability to view and change every step code and validate the results. Also, many used algorithms are too outdated and produce terrible results like to tension surfaces in GMT which is used widely in GMTSAR (hmm, how about to control a smothness of derivative? Tension surfaces were invented when all the Earths computers where less powerfull than your smarthone today. If you are interested I shared the examples in GMTSAR bug tracker). Anyway, I found GMT mathematics is really crazy and the developers replace one incorrect algorithm by another and back often as we see in the codes and the changelog). That was enough reason to check all the used algorithms and replace these by modern and correct ones. By this way, I use only GMTSAR C codes with my patches to fix some errors and allow interoperability with Python wrappers plus my own codes around them. GMTSAR codes are fine and without crazy GMT codes work better and the processing is much faster. Alright, I spent one month to make the initial PyGMTSAR realization and it works. Recently, I returned to the project to add some more sophisticated features like to scenes and subswathes stitching. I'm going to share some of my geological exprorations and seismic models as live examples on Google Colab as soon as it will be possible to do. How lineaments and ore zones are related to interferograms? How gas and oil deposits are related to surface movements on interferometry displacement maps? I have the answer and I work on the tools to model and visualize them.

You'd find my theoretical models and processing codes foir geophisical inversions in Github repository https://github.com/mobigroup/gis-snippets and tools for the 4D results vizualization in https://github.com/mobigroup/ParaView-plugins

Ah yes, a little bit about me. I have STEM master's degree in radio physics and in 2004 I was awarded first prize of the All-Russian Physics competition for significant results in Inverse modeling for non-linear optics and holography, also applicable for Inverse Modeling of Gravity, Magnetic, and Thermal fields. In addition to my fundamental science knowledge, I’m world class data scientist and software developer with 20 years experience in science and industrial development. I have worked on government contracts and universities projects and on projects for LG Corp, Google Inc, etc. You are able to find some of my software and results on LinkedIn and GitHub and Upwork. By the way, I left Russia many years ago and I work remotely for about 20 years.

### To order some research, development and support see my profile on freelance platform [Upwork](https://www.upwork.com/freelancers/~01e65e8e7221758623)

### @ Alexey Pechnikov, August, 2022

[Geological models on YouTube channel](https://www.youtube.com/channel/UCSEeXKAn9f_bDiTjT6l87Lg)

[Augmented Reality (AR) Geological Models](https://mobigroup.github.io/ParaView-Blender-AR/)

[GitHub repositories](https://github.com/mobigroup)

[English posts and articles on LinkedIn](https://www.linkedin.com/in/alexey-pechnikov/)

[Russian articles on Habr](https://habr.com/ru/users/N-Cube/posts/)

$\large\color{blue}{\text{Hint: Use menu Cell} \to \text{Run All or Runtime} \to \text{Complete All or Runtime} \to \text{Run All}}$
$\large\color{blue}{\text{(depending of your localization settings) to execute the entire notebook}}$

## Load Modules to Check Environment

In [ ]:
import platform, sys, os

## Debian 10 and Google Colab GMTSAR Installation

### On Google Cloud AI Notebooks: check root access

On Google Cloud AI Notebooks sometimes we have an issue when "sudo" requires a password. In this case drop the instance and create a new one and - that's important - wait 5-10 minutes before connect to it using link "OPEN JUPYTERLAB"

In [ ]:
if platform.system() == 'Linux':
    !sudo date

### Install https://github.com/mobigroup/gmtsar

I make lots of changes on GMTSAR C-coded tools and some of them are not merged to the upstream GMTSAR yet because all the patches need to be validated and discussed before. Also, my Python extensions are provided in my GMTSAR fork only. I hope in the future to provide a standalone python packager as wrapper around upstream GMTSAR but there is a long way to it.

In [ ]:
if platform.system() == 'Linux':
    count = !ls /usr/local | grep GMTSAR | wc -l
    if count == ['0']:
        !apt install -y csh autoconf gfortran \
            libtiff5-dev libhdf5-dev liblapack-dev libgmt-dev gmt-dcw gmt-gshhg gmt > /dev/null
        !cd /usr/local && git clone --branch master https://github.com/mobigroup/gmtsar GMTSAR > /dev/null
        !cd /usr/local/GMTSAR && autoconf > /dev/null
        !cd /usr/local/GMTSAR && ./configure --with-orbits-dir=/tmp > /dev/null
        !cd /usr/local/GMTSAR && make 1>/dev/null 2>/dev/null
        !cd /usr/local/GMTSAR && make install >/dev/null

## Define ENV Variables for Jupyter Instance

In [ ]:
# use default GMTSAR installation path
GMTSAR = '/usr/local/GMTSAR'
PATH = os.environ['PATH']

if PATH.find('GMTSAR') == -1:
    PATH = os.environ['PATH'] + f':{GMTSAR}/bin/'
    %env PATH {PATH}
    %env GMTSAR {GMTSAR}

## Install Python Modules

Maybe you need to restart your notebook, follow the instructions printing below.

The installation takes a long time on fresh Debian 10 and a short time on Google Colab

In [ ]:
!{sys.executable} --version

In [ ]:
if platform.system() == 'Linux':
    !{sys.executable} -m pip install cartopy==0.19.0.post1 1>/dev/null 2>/dev/null
    !{sys.executable} -m pip install xarray==0.19.0        1>/dev/null 2>/dev/null
    !{sys.executable} -m pip install scipy==1.7.1          1>/dev/null 2>/dev/null

In [ ]:
if platform.system() == 'Linux':
    !{sys.executable} -m pip install \
        h5py netcdf4 h5netcdf \
        rasterio rioxarray numpy \
        scikit-image scipy sklearn \
        xarray dask distributed zarr nc-time-axis \
        pandas geopandas \
        sentineleof elevation \
        matplotlib seaborn geoviews hvplot datashader bokeh \
        xmltodict joblib tqdm 1>/dev/null 2>/dev/null

$\large\color{red}{\text{Attention: On Google Colab we need to restart kernel once when modules installed}}$

$\large\color{blue}{\text{Hint: Use menu Cell} \to \text{Run All or Runtime} \to \text{Complete All or Runtime} \to \text{Run All}}$
$\large\color{blue}{\text{(depending of your localization settings) to execute the entire notebook}}$

In [ ]:
if platform.system() == 'Linux':
    import xarray
    import time
    print (xarray.__version__)
    if xarray.__version__ != '0.19.0':
        print ("""
    ***********************************************************************************
    *
    Do not worry, runtime is stopped by design. Please run the notebook again.
    *
    ***********************************************************************************
    """)
        time.sleep(1)
        os.kill(os.getpid(), 9)

## Load and Setup Python Modules

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
# supress numpy warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# plotting modules
import hvplot.pandas  # noqa
import hvplot.xarray  # noqa
import holoviews as hv
pd.options.plotting.backend = 'hvplot'
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [ ]:
# define Pandas display settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

gstiles = hv.Tiles('https://mt1.google.com/vt/lyrs=s&x={X}&y={Y}&z={Z}', name='Google Satellite')
ottiles = hv.Tiles('https://tile.opentopomap.org/{Z}/{X}/{Y}.png', name='Open Topo')

## Load Custom Python Modules

In [ ]:
sys.path.append(os.path.join(os.environ['GMTSAR'],'gmtsar', 'py'))

from PRM import PRM
from SBAS import SBAS

## Define 3 Sentinel-1 Scenes and Processing Parameters

When you need more scenes and SBAS analysis  see examples on PyGMTSAR GitHub page https://github.com/mobigroup/gmtsar

### Descending Orbit Configuration

In [ ]:
#SCENES = ['S1A_IW_SLC__1SDV_20171112T030148_20171112T030216_019226_0208EC_EC55',
#          'S1B_IW_SLC__1SDV_20171118T030054_20171118T030121_008330_00EBEB_FE01',
#          'S1B_IW_SLC__1SDV_20171118T030119_20171118T030146_008330_00EBEB_2F6B']
#ORBIT        = 'D'
#SUBSWATH     = 1

### Ascending Orbit Configuration

In [ ]:
SCENES = ['S1A_IW_SLC__1SDV_20171111T150004_20171111T150032_019219_0208AF_EE89',
          'S1B_IW_SLC__1SDV_20171117T145900_20171117T145928_008323_00EBAB_B716',
          'S1B_IW_SLC__1SDV_20171117T145926_20171117T145953_008323_00EBAB_AFB8']
ORBIT        = 'A'
SUBSWATH     = 3

In [ ]:
WORKDIR      = 'raw'
DATADIR      = 'data'
DEMFILE      = None
POLARIZATION = 'VV'
CORRLIMIT    = 0.10
DEFOMAX      = 0

## Download and Unpack Datasets (Optional)

### Input ASF (Earthdata) User and Password

When data directory does not exists or empty the Sentinel-1 scenes downloaded from Alaska Satellite Facility (ASF) datastore. Use your Earthdata Login credentials. If you do not have a Earthdata Login, create one at https://urs.earthdata.nasa.gov//users/new

In [ ]:
if os.path.isdir(DATADIR) and len([fname for fname in os.listdir(DATADIR) if fname[0]!='.'])>0:
    print ('Data directory already exists and it is not empty, miss downloading')
else:
    import time
    import getpass
    print ('Data directory does not exists or it is empty, download the dataset')
    # to print the message BEFORE the usename and password fields
    time.sleep(1)
    username = getpass.getpass('Please enter your ASF username and press Enter key:')
    password = getpass.getpass('Please enter your ASF password and press Enter key:')
    # download the scenes from ASF and unpack only required files to data directory
    for scene in SCENES:
        url = f'https://datapool.asf.alaska.edu/SLC/S{scene[2:3]}/{scene}.zip'
        !wget --user={username} --password={password} -qc {url}
        for ext in {'xml', 'tiff'}:
            for subswath in str(SUBSWATH):
              !unzip -j -n {scene}.zip '*.SAFE/*/s1?-iw{subswath}-slc-{POLARIZATION.lower()}-*-*.{ext}' -d {DATADIR}

## Init SBAS

Search recursively for measurement (.tiff) and annotation (.xml) and orbit (.EOF) files in the DATA directory. It can be directory with full unzipped scenes (.SAFE) subdirectories or just a directory with the list of pairs of required .tiff and .xml files (maybe pre-filtered for orbit, polarization and subswath to save disk space). If orbit files and DEM are missed these will be downloaded automatically below.

### Reuse Backup Dataset If Exists (uncomment the code below if needed)

When the full dataset is already downloaded and processing `SBAS.backup()` command allows to save the-preprocessed stitched and cropped scenes and DEM and orbit files into the specified directory and reuse the dataset later. By this way you can process a big dataset on cloud environment for the first time and download the pre-processed small dataset to continue work on your local computer or on Google Colab.

### Select Original Secenes and Download DEM and Orbits Later

Use filters to find required subswath, polarization and orbit in original scenes .SAFE directories in the data directory.

In [ ]:
sbas = SBAS(DATADIR, DEMFILE, basedir=WORKDIR,
            filter_orbit=ORBIT,
            filter_subswath=SUBSWATH,
            filter_polarization=POLARIZATION)

In [ ]:
sbas.to_dataframe()

### Download Sentinel-1 Orbits

The function below downloads orbit files.
Besides, for faster processing we can automatically use pre-downloaded orbit files in data directory.

In [ ]:
# also, we can place pre-downloaded orbit files to data directory 
sbas.download_orbits()

### Show the Dataset

All the orbit files found, see 'orbitpath' column

In [ ]:
sbas.to_dataframe()

## Additional Data Filtering

All the scenes stored internally as a Pandas DataFrame "df" inside the SBAS object. So we are able to use Pandas operations on the sbas.df for additional scenes filtering and so on. Maybe, do you need to select just a few of scenes for a selected date range? In this case check the code below and adopt it for you.

In [ ]:
# select scenes for the date range
#sbas.df = sbas.df.query('"2017-11-10" <= index <= "2017-11-20"')
#sbas.df

In [ ]:
# select first 2 scenes only
#sbas.df = sbas.df.head(2)
#sbas.df

## Reframe Scenes (Optional)

Stitch sequential scenes and crop the subswath to a smaller area for faster processing when the full area is not needed.

### Define Pair of Pins for Each Subswath (Optional)

To create a stitched and maybe cropped frame two pins should be defined. Use the plots below to define the pins coordinates. Hint: in case of an error you see the notes how to fix the issues and you are still able to map the pins using the code below (anyway the issues should be fixed to continue the calculations).

This step can be missed and for this case automatically generated pins cover the full master image.

In [ ]:
# just miss the function call or run it without arguments to auto-pinning
sbas.set_pins([46, 34.6, None, None])

### Static Plot

In [ ]:
plt.figure(figsize=(8,4), dpi=150)
gdf =sbas.to_dataframe()
cmap = plt.cm.get_cmap('bwr', len(gdf.index.unique()))
colors = dict([(v,cmap(k)) for k,v in enumerate(gdf.index.unique())])
gdf.reset_index().plot(color=[colors[k] for k in gdf.index], alpha=0.2, edgecolor='black', ax=plt.gca())
plt.scatter(x=sbas.get_pins()[::2], y=sbas.get_pins()[1::2], color='black', s=10)
plt.title('Sentinel1 Scenes Approximate Location', fontsize=12)
plt.show()

### Interactive Plot

Be careful because interactive plots require more RAM to be visualized

The plots below do not work on Debian 10 and Python 3.7

In [ ]:
plots = None
if platform.system() == 'Darwin':
    opts_common = {'x':'lon', 'y':'lat', 'geo':True, 'width':800, 'height':600}
    opts1 = {'tiles':gstiles, 'alpha':0.4, 'title': 'Sentinel1 Scenes Approximate Location'}
    plots = sbas.to_dataframe().reset_index().hvplot(color='date', **{**opts_common, **opts1}) \
        * pd.DataFrame({'x': sbas.get_pins()[::2], 'y': sbas.get_pins()[1::2]}).hvplot.points(y='y', x='x', c='black', geo=True)
plots

### Crop the Frame (Optional)

When we satisfied with the pins positions and the target frame we are ready to crop the frame.
Just comment the code cell below when you don't need to crop the area.

In [ ]:
sbas.reframe_parallel()

### Show the updated dataset

The newly created scene frames .tiff and .xml files plus orbit files .EOF can be copied for the future processing using `SBAS.backup()` function to use them instead of the original full scenes.

In [ ]:
sbas.to_dataframe()

### Download SRTM DEM

The function below downloads SRTM1 or SRTM3 DEM and converts heights to ellipsoidal model using EGM96 grid.
Besides, for faster processing we can use pre-defined DEM file as explained above. Select product=SRTM1 for 30m resolution and product=SRTM3 for 90m resolution SRTM DEM.

In [ ]:
if platform.system() == 'Darwin':
    # this command allows to download DEM for a large area but it requires GMT 6.1+ and it does not work on Google Colab
    sbas.download_dem(backend='GMT')
else:
    # for old Linux Debian and Ubuntu use default backend to download SRTM3 DEM for large areas (4°x4° and more)
    #sbas.download_dem(product='SRTM3')
    # for old Linux Debian and Ubuntu use default backend to download SRTM1 DEM for small areas (less than 4°x4°)
    sbas.download_dem(product='SRTM1')

### Static Plots

In [ ]:
plt.figure(figsize=(12,4), dpi=150)
dem = sbas.get_dem()[::4,::4]
dem.plot.imshow(cmap='gray', vmin=dem.min(), vmax=dem.max())
sbas.to_dataframe().plot(color='orange', alpha=0.2, ax=plt.gca())
plt.title('Sentinel1 Frame on DEM', fontsize=18)
plt.show()

### Interactive Plot

Be careful because interactive plots require more RAM to be visualized

The plots below do not work on Debian 10 and Python 3.7

In [ ]:
plots = None
if platform.system() == 'Darwin':
    title = 'Subswath Topo'
    plots = sbas.get_dem()[::4,::4].hvplot(invert=False, cmap='gray', alpha=1, title=title) \
        * sbas.to_dataframe().reset_index().hvplot(color='date', alpha=0.2)
plots

## Align a pair of Images

In [ ]:
sbas.stack_parallel()

## DEM in Radar Coordinates

In [ ]:
sbas.topo_ra_parallel()

### Load Grids

The grids are NetCDF files processing as xarray DataArrays.

In [ ]:
topo_ra = sbas.open_grid('topo_ra')

### Static Plot

In [ ]:
plt.figure(figsize=(12,4), dpi=300)
topo_ra[::4,::4].plot.imshow(cmap='gray', vmin=0)
plt.xlabel('Range', fontsize=16)
plt.ylabel('Azimuth', fontsize=16)
plt.title('Topography in Radar Coordinates', fontsize=18)
plt.show()

### Interactive Plot

Be careful because interactive plots require more RAM to be visualized

The plots below do not work on Debian 10 and Python 3.7

In [ ]:
plots = None
if platform.system() == 'Darwin':
    plot_opts = {'rasterize': True, 'xlabel':'Range', 'ylabel':'Azimuth', 'width':500, 'height':400}
    plots = topo_ra[::4,::4].hvplot(cmap='gray', title='Topography in Radar Coordinates', **plot_opts)
plots

## Interferogram

Define a single interferogram or a SBAS series. Make direct and reverse interferograms (from past to future or from future to past).

Decimation is useful to save disk space. Geocoding results are always produced on the provided DEM grid so the output grid and resolution are the same to the DEM. By this way, ascending and descending orbit results are always defined on the same grid by design. The internal processing cells is about 15 x 15 meters size and for default output 60m resolution (like to GMTSAR and GAMMA software) decimation 4x4 is reasonable. For the default wavelength=200 for Gaussian filter 1/4 of wavelength is approximately equal to ~60 meters and better resolution is mostly useless (while it can be used for small objects detection). For wavelength=400 meters use 90m or 120m DEM resolution with decimation 6x6 or 8x8 and for wavelength=100 meters use decimation 2x2.

The grids are NetCDF files processing as xarray DataArrays.

In [ ]:
# for a pair of scenes only two interferograms can be produced
# this one is selected for scenes sorted by the date in direct order
pairs = [sbas.to_dataframe().index]

In [ ]:
# miss optional "func" argument when post-processing is not required
# define a postprocessing function for decimation as func=decimator
# for 30m DEM use decimation x=2, y=2 (is equal to GMTSAR rdec=8, adec=2) or 4x4 for 60m DEM
decimator = lambda dataarray: dataarray.coarsen({'y': 4, 'x': 4}, boundary='trim').median()

# default parameters: wavelength=200, psize=32, func=None (no postprocessing)
sbas.intf_parallel(pairs, func=decimator)

### Load Grids

The grids can be cropped automatically to drop empty areas around valid area (use crop_valid=True). On-the fly geocoding from radar to geographic coordinates is possible by geocode=True.

The grids are NetCDF files in radar coordinates processing as xarray DataArrays. For a single interferogramm processing convert the 3D stack to 2D grid using index [0].

In [ ]:
phasefilt = sbas.open_grids(pairs, 'phasefilt', geocode=True, crop_valid=True)[0]
corr = sbas.open_grids(pairs, 'corr', geocode=True, crop_valid=True)[0]

### Static Plots

In [ ]:
plt.figure(figsize=(12,4), dpi=300)
phasefilt.plot.imshow(vmin=-np.pi, vmax=np.pi, cmap='gist_rainbow_r')
plt.title('Phase, [rad]', fontsize=18)
plt.show()

In [ ]:
plt.figure(figsize=(12,4), dpi=300)
corr.plot.imshow(vmin=-0, vmax=1, cmap='gray')
plt.title('Correlation', fontsize=18)
plt.show()

### Interactive Plot

Be careful because interactive plots require more RAM to be visualized

The plots below do not work on Debian 10 and Python 3.7

In [ ]:
plots = None
if platform.system() == 'Darwin':
    opts_common = {'x':'lon', 'y':'lat', 'geo':True, 'width':800, 'height':600, 'colorbar':True, 'padding':0}
    opts1 = {'tiles':gstiles, 'alpha':0.4, 'cmap':'gist_rainbow_r', 'clim':(-np.pi, np.pi),
            'title': f'Sentinel-1 Interferogram [rad] Co-Seismic Pair {pairs[0][0]} - {pairs[0][1]}'}
    plots = phasefilt.hvplot(**{**opts_common, **opts1})
plots

## Unwrapping

Unwrapping process requires a lot of RAM and that's really RAM consuming when a lot of parallel proccesses running togeter. To limit the parallel processing tasks apply argument "n_jobs". The default value n_jobs=-1 means all the processor cores van be used. Also, use interferogram decimation above to produce smaller interferograms. And in addition a custom SNAPHU configuration can reduce RAM usage as explained below.

Attention: in case of crash on MacOS Apple Silicon run Jupyter as

`OBJC_DISABLE_INITIALIZE_FORK_SAFETY=YES no_proxy='*' jupyter notebook`

### Unwrapping Post-Processing

Define post-processing function to exclude low-coherence areas and maybe fill them by nearest neighbour interpolation.

In [ ]:
# define a post-processing function to crop low-coherence areas
cleaner = lambda corr, unwrap: xr.where(corr>=CORRLIMIT, unwrap, np.nan)

# define a post-processing function to crop and interpolate low-coherence areas
#cleaner = lambda corr, unwrap: sbas.nearest_grid(xr.where(corr>=CORRLIMIT, unwrap, np.nan))

### Multiple Interferograms Parallel Unwrapping

### Single Interferogram Parallel Unwrapping

SNAPHU unwrapper allows to split large scene to tiles for parallel processing and accurately enough merge the tiles to a single image. That's especially helpful to unwrap a single interferogram using all the processor cores and save RAM consumption drastically.

### Generate Custom SNAPHU Config

Default SNAPHU configuration allows to run multiple unwrapping tasks in parallel. At the same time, for a single interferogram processing or large interferograms unwrapping SNAPHU tiling can decrease the both processing time and RAM comsumption. Be careful to prevent unconnected zones on the tiles when a high coherency area is fully arounded by low-coherent buffer on some tiles (redefine tiles or increase tiles row and column overlapping parameters ROWOVRLP and COLOVRLP).

In [ ]:
# use NPROC parameter to define number of cores for the processing, by default all the cores used
conf = sbas.PRM().snaphu_config(defomax=DEFOMAX, NTILEROW=1, NTILECOL=8, ROWOVRLP=200, COLOVRLP=200)

In [ ]:
# check the generated SNAPHU config and tune the SNAPHU parameters above if needed.
print ('SNAPHU custom config generated:')
print (conf)

In [ ]:
# "FileNotFoundError" SNAPHU error means the processing is not completed normally for some tiles
# try again with different SNAPHU parameters

# miss "func" argument or set it to None when post-processing is not required
# run only one unwrapping task at the time because the custom SNAPHU configuration already uses all the cores
sbas.unwrap_parallel(pairs, n_jobs=1, threshold=CORRLIMIT, func=cleaner, conf=conf)

### Load Grid

The grids can be cropped automatically to drop empty areas around valid area. Use `open_grids(..., crop_valid=True)` to enable the auto cropping feature. Argument geocode=True means on-the-fly geocoding from radar to geographic coordinates (all the grids saved on disk in radar coordinates plus geocoding matrices are generated for the fast geocoding). For a single interferogramm processing we convert the 3D stack to 2D grid using index [0].

In [ ]:
unwrap = sbas.open_grids(pairs, 'unwrap', geocode=True, crop_valid=True)[0]

### Static Plot

In [ ]:
plt.figure(figsize=(12,4), dpi=300)
zmin, zmax = np.nanquantile(unwrap, [0.01, 0.99])
unwrap.plot.imshow(vmin=zmin, vmax=zmax, cmap='jet')
plt.title('Unwrapped Phase, [rad]', fontsize=18)
plt.show()

### Interactive Plot

Be careful because interactive plots require more RAM to be visualized

The plots below do not work on Debian 10 and Python 3.7

In [ ]:
plots = None
if platform.system() == 'Darwin':
    opts_common = {'x':'lon', 'y':'lat', 'geo':True, 'width':800, 'height':600, 'colorbar':True, 'padding':0}
    opts2 = {'tiles':gstiles, 'alpha':0.5, 'cmap':'turbo', 'clim':(zmin, zmax),
             'title': f'Sentinel-1 Unwrapped Phase [rad] Co-Seismic Pair {pairs[0][0]} - {pairs[0][1]}'}
    plots = unwrap.hvplot(**{**opts_common, **opts2})
plots

## LOS Displacement

### Load and Calculate Grid

The grids can be post-processed using user-defined function "func". SBAS function `los_displacement_mm()` converts unwrapped phase values to LOS displacement in millimeters. For a single interferogramm processing convert the 3D stack to 2D grid using index [0].

In [ ]:
los_disp_mm = sbas.open_grids(pairs, 'unwrap', func=sbas.los_displacement_mm, geocode=True, crop_valid=True)[0]

### Static Plot

In [ ]:
plt.figure(figsize=(12,4), dpi=300)
zmin, zmax = np.nanquantile(los_disp_mm, [0.01, 0.99])
los_disp_mm.plot.imshow(vmin=zmin, vmax=zmax, cmap='jet')
plt.title('LOS Displacement, [mm]', fontsize=18)
plt.show()

### Interactive Plot

Be careful because interactive plots require more RAM to be visualized

The plots below do not work on Debian 10 and Python 3.7

In [ ]:
plots = None
if platform.system() == 'Darwin':
    opts_common = {'x':'lon', 'y':'lat', 'geo':True, 'width':800, 'height':600, 'colorbar':True, 'padding':0}
    opts2 = {'tiles':gstiles, 'alpha':0.5, 'cmap':'turbo', 'clim':(zmin, zmax),
             'title': f'Sentinel-1 LOS Displacement [mm] Co-Seismic Pair {pairs[0][0]} - {pairs[0][1]}'}
    plots = los_disp_mm.hvplot(**{**opts_common, **opts2})
plots

## Measure Maximum and Minimum LOS Displacement, [mm]

In [ ]:
los_disp_mm.min().round().item(), los_disp_mm.max().round().item()

## Compare Results to GMTSAR, GAMMA, SNAP

* PyGMTSAR results full subswath 3 for LOS displacement depends a bit of the unwrapping method above:

  - 85.8 cm up and 15 cm down with amplitude 100.8 cm for a single-image unwrapping,
  - 87.5 cm up and 15 cm down with amplitude 102.5 cm for tiled unwrapping.

Pay attention for the cropped subswath the results are different (83 cm up and 23 cm down). Run the notebook without pinning to produce the full subswath results.

* GAMMA software package results: [Co-Seismic Deformation and Fault Slip Model of the 2017 Mw 7.3 Darbandikhan, Iran–Iraq Earthquake Inferred from D-InSAR Measurements](https://www.mdpi.com/2072-4292/11/21/2521/htm). The reported amplitude is 84 cm + 15 cm = 99 cm and it's very close to PyGMTSAR measured 100.8 cm for single-image unwrapping:

> for the ascending orbit the maximum value in the southwestern lobe of about 84 cm,
> and the minimum value in the northeastern lobe along the line-of-sight (LOS) of 15 cm.


* Ustream GMTSAR results: [Surface Displacements of the 12 November 2017 Iran – Iraq Earthquake derived using SAR Interferometry](https://www.researchgate.net/publication/333108916_Surface_Displacements_of_the_12_November_2017_Iran_-_Iraq_Earthquake_derived_using_SAR_Interferometry) The reported amplitude is 80 cm + 22 cm = 102 cm and it's almost equal to PyGMTSAR measured 102.5 cm for tiled unwrapping:

> The coseismic LOS displacements corresponding to the uplift and subsidence are observed to be ... +0.80 m and -0.22 m for the Sentinel-1 ascending

* SNAP processing instructions and output interferogram: [Creating Interferogram for Mapping Earthquake Deformation by using Sentinel-1 Data in SNAP](https://dges.carleton.ca/CUOSGwiki/index.php/Creating_Interferogram_for_Mapping_Earthquake_Deformation_by_using_Sentinel-1_Data_in_SNAP) While there is no unwrapping processing here we are able to compare the interferograms, see plots below.

### Download SNAP Interferogram

In [ ]:
url = "https://dges.carleton.ca/CUOSGwiki/images/thumb/b/b1/Step12_II.JPG/1400px-Step12_II.JPG"
!wget -qc {url}

### Static Plot

In [ ]:
f, (ax1, ax2) = plt.subplots(1,2,figsize=(12,4), dpi=300)
phasefilt.plot.imshow(vmin=-np.pi, vmax=np.pi, cmap='gist_rainbow_r', ax=ax1)
ax1.set_title('PyGMTSAR IW3 Phase, [rad]', fontsize=18)

from skimage import io
img = io.imread('1400px-Step12_II.JPG')
ax2.imshow(img)
ax2.axis('off')
ax2.set_title('SNAP Cropped IW3 Phase, [rad]', fontsize=18)

plt.show()

### Interactive Plot

Be careful because interactive plots require more RAM to be visualized

The plots below do not work on Debian 10 and Python 3.7

In [ ]:
plots = None
if platform.system() == 'Darwin':
    image = hv.RGB.load_image('1400px-Step12_II.JPG')
    opts_common = {'x':'lon', 'y':'lat', 'geo':True, 'width':500, 'height':400, 'colorbar':True, 'padding':0}
    opts1 = {'tiles':gstiles, 'alpha':0.4, 'cmap':'gist_rainbow_r', 'clim':(-np.pi, np.pi),
            'title': f'Sentinel-1 Interferogram [rad] Co-Seismic Pair'}
    opts2 = {'tiles':gstiles, 'alpha':0.4, 'title': 'Sentinel1 Scenes'}
    plots = phasefilt.hvplot(**{**opts_common, **opts1}) \
        + image.options(width=500, height=400, title='SNAP Cropped IW3 Phase, [rad]') \
        + sbas.to_dataframe().reset_index().hvplot(color='date', **{**opts_common, **opts2}) \
            * pd.DataFrame({'x': sbas.get_pins()[::2], 'y': sbas.get_pins()[1::2]}).hvplot.points(y='y', x='x', c='black', geo=True)
    plots = hv.Layout(plots).cols(2)
plots

## Save the Results

Save the results in geospatial data formats like to NetCDF, GeoTIFF and others. The both formats (NetCDF and GeoTIFF) can be opened in QGIS and other GIS applications.

In [ ]:
# define the filenames for phase, correlation, LOS displacement
phasefilt_filename = f'phasefilt_iw{SUBSWATH}_{POLARIZATION}_{ORBIT}.nc'
corr_filename = f'corr_iw{SUBSWATH}_{POLARIZATION}_{ORBIT}.nc'
displacement_filename = f'los_disp_mm_iw{SUBSWATH}_{POLARIZATION}_{ORBIT}.nc'

# save the results
phasefilt.to_netcdf(phasefilt_filename)
corr.to_netcdf(corr_filename)
los_disp_mm.to_netcdf(displacement_filename)

### Export from Google Colab 

In [ ]:
try:
    from google.colab import files
    files.download(phasefilt_filename)
    files.download(corr_filename)
    files.download(displacement_filename)
except:
    pass

## Backup Pre-Processed Data

Save the stitched and cropped scenes and downloaded DEM and orbit files to a new directory using `SBAS.backup(dir)` command and load the dataset later as `SBAS(dir, 'dir/DEM_WGS84.nc', basedir=WORKDIR)` Do not forgot to download the backup to use it later.

In [ ]:
if DATADIR != 'backup':
    print ('Create backup to use it for the next reproducible runs without the original scenes downloading')
    sbas.backup('backup')

In [ ]:
!ls backup

## Conclusion

For now you have the full control on interferometry processing and unwrapping and able to run it everywhere: on free of charge Google Colab instances, on local MacOS and Linux computers and on Amazon EC2 and Google Cloud VM and AI Notebook instances.